Import library and data

In [215]:
import torch, torchvision, os, time
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_dataset = torchvision.datasets.MNIST(root = os.path.join(os.getcwd(),'MNIST_data'),
                                            train = True,
                                            download =True,
                                            transform = transforms.ToTensor())

test_dataset = torchvision.datasets.MNIST(root = os.path.join(os.getcwd(),'MNIST_data'),
                                            train = False,
                                            transform = transforms.ToTensor())                                     

loader_batch_size = 100
train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                                batch_size=loader_batch_size,
                                                shuffle = True)
test_dataloader = torch.utils.data.DataLoader(test_dataset,
                                                batch_size=loader_batch_size,
                                                shuffle = False)

# acc function and nn model

In [216]:
class NNlinear(nn.Module):
    def __init__(self,input_size,hidden_layer1_size,hidden_layer2_size,output_size):
        super(NNlinear,self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_layer1_size,bias=True)
        self.layer2 = nn.Linear(hidden_layer1_size,hidden_layer2_size,bias=True)
        self.output = nn.Linear(hidden_layer2_size,output_size)

    def forward(self,x):
        z = torch.relu(self.layer1(x))
        z = torch.relu(self.layer2(z))
        z = torch.sigmoid(self.output(z))
        return z
def accuracy_test(model_ML,dataloader_model):
    z_val = model_ML(dataloader_model.dataset.data.view(-1,28*28).type(torch.float32))
    acc_val = (((z_val.argmax(dim=1)==dataloader_model.dataset.targets)*1.0).mean()*100).item()
    return acc_val

# trainning

In [217]:
pre_trainned = True
model = NNlinear(28*28,50,20,10)
NN_file = 'NN_model_state_dict.pt'
if pre_trainned:
    model.load_state_dict(torch.load(NN_file))
loss = nn.CrossEntropyLoss()
model.eval()
optimizer = torch.optim.Adam(model.parameters(),lr = 1e-3)

max_epoch = int(1e3)
with torch.no_grad():
    acc_new = accuracy_test(model,train_dataloader)
    acc_test = accuracy_test(model,test_dataloader)
print(f'Epoch 0, acc train = {acc_new:.4f}%, acc test = {acc_test:.4f}%')
for ep in range(max_epoch):
    for img,label in train_dataloader:
        z = model(img.view(-1,28*28))
        J = loss(z,label)
        J.backward()
        optimizer.step()
        optimizer.zero_grad()
    with torch.no_grad():
        acc_new = accuracy_test(model,train_dataloader)
        acc_test = accuracy_test(model,test_dataloader)
    print(f'Epoch {ep+1} loss = {J:.4f}, acc_train = {acc_new:.4f}%, acc test = {acc_test:.4f}%')
    if np.abs((acc_new-acc)/acc)*100 < 1:
        break
    else:
        acc = acc_new
torch.save(model.state_dict(),NN_file)

Epoch 0, acc train = 9.3483%, acc test = 8.7200%
Epoch 1 loss = 1.5498, acc_train = 81.1200%, acc test = 80.9900%
Epoch 2 loss = 1.5299, acc_train = 87.1083%, acc test = 87.3200%
Epoch 3 loss = 1.5383, acc_train = 90.2350%, acc test = 89.8800%
Epoch 4 loss = 1.5049, acc_train = 91.8450%, acc test = 91.0200%
Epoch 5 loss = 1.4988, acc_train = 91.2933%, acc test = 90.9400%
